In [ ]:
import os
import sys
import shutil

sys.path.append('../..')
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from ase import Atoms
from ase.optimize import BFGS
from ase.vibrations import Vibrations
from Intra import Intra
from Utilities import show_results, write_xyz
from Constants import EV_TO_KJ, H_TO_KJ, BOHR_TO_ANGSTROM

from rdkit.Chem.rdmolfiles import MolFromXYZFile

In [ ]:
DATA_ICE13 = np.load('../../../data/test_sets/ICE13/ICE13.npy', allow_pickle=True).item()
DATA_X23 = np.load('../../../data/test_sets/x23_mini.npy', allow_pickle=True).item()
KEYS = ['CYHEXO', 'ACETAC07', 'ADAMAN08', 'ANTCEN09', 'BENZEN01', 'CYANAM01', 
        'CYTSIN01', 'ECARBM01', 'FORMAM02', 'HXMTAM10', 'IMAZOL04', 'NAPHTA23', 
        'OXALAC03', 'OXALAC04', 'PYRAZI01', 'PYRZOL05', 'TRIZIN', 'TROXAN11', 
        'SUCACB03', 'URACIL', 'UREAXX12', 'nh3', 'co2']
DATA = {**DATA_X23, **DATA_ICE13}

In [ ]:
MONOMER_ENERGIES = {}
Intra = Intra(use_D3=True)

In [ ]:
for key in KEYS:
    print(key)
    if key not in ['co2', 'OXALAC03', 'OXALAC04']:
        coords_monomer = DATA_X23[key]['monomer_coordinates'][None]
        elements_monomer = DATA_X23[key]['monomer_symbols']
        geometry = by.Geometry(elements_monomer, coords_monomer[0])
        optimizer = by.Berny(geometry, debug=True, maxsteps=50, dihedral=True)
        Intra.initialize(coords_monomer, elements_monomer)
        for idg, geom in enumerate(optimizer):
            coords = geom.coords[None].astype(np.float32)
            V_intra, dV_intra, (monos, dipos, quads, ratio) = Intra(coords, grad=True)
            debug = optimizer.send((V_intra.numpy()[0, 0] / H_TO_KJ, dV_intra.numpy()[0] / (H_TO_KJ / BOHR_TO_ANGSTROM)))
            print(f'Step - {idg}')
            print(V_intra.numpy()[0, 0])
        MONOMER_ENERGIES[key] = V_intra.numpy()[0, 0]

In [ ]:
rdkit_mol = MolFromXYZFile('data/oxalac_min.xyz')
coords_monomer = rdkit_mol.GetConformer(0).GetPositions()[None]
elements_monomer = [a.GetSymbol() for a in rdkit_mol.GetAtoms()]
geometry = by.Geometry(elements_monomer, coords_monomer[0])
optimizer = by.Berny(geometry, debug=True, maxsteps=50, dihedral=True)
Intra.initialize(coords_monomer, elements_monomer)
for idg, geom in enumerate(optimizer):
    coords = geom.coords[None].astype(np.float32)
    V_intra, dV_intra, (monos, dipos, quads, ratio) = Intra(coords, grad=True)
    debug = optimizer.send((V_intra.numpy()[0, 0] / H_TO_KJ, dV_intra.numpy()[0] / (H_TO_KJ / BOHR_TO_ANGSTROM)))
    print(f'Step - {idg}')
    print(V_intra.numpy()[0, 0])
MONOMER_ENERGIES['OXALAC03'] = V_intra.numpy()[0, 0]
MONOMER_ENERGIES['OXALAC04'] = V_intra.numpy()[0, 0]

In [ ]:
key = 'Ih'
coords_monomer = DATA_ICE13[key]['monomer_coordinates'][None]
elements_monomer = DATA_ICE13[key]['monomer_symbols']
geometry = by.Geometry(elements_monomer, coords_monomer[0])
optimizer = by.Berny(geometry, debug=True, maxsteps=50, dihedral=True)
Intra.initialize(coords_monomer, elements_monomer)
for idg, geom in enumerate(optimizer):
    coords = geom.coords[None].astype(np.float32)
    V_intra, dV_intra, (monos, dipos, quads, ratio) = Intra(coords, grad=True)
    debug = optimizer.send((V_intra.numpy()[0, 0] / H_TO_KJ, dV_intra.numpy()[0] / (H_TO_KJ / BOHR_TO_ANGSTROM)))
    print(f'Step - {idg}')
    print(V_intra.numpy()[0, 0])
MONOMER_ENERGIES['ICE'] = V_intra.numpy()[0, 0]

In [ ]:
key = 'co2'
coords_monomer = DATA_X23[key]['monomer_coordinates'][None]
elements_monomer = DATA_X23[key]['monomer_symbols']
geometry = by.Geometry(elements_monomer, coords_monomer[0])
Intra.initialize(coords_monomer, elements_monomer)

V_intra, dV_intra, (monos, dipos, quads, ratio) = Intra(coords_monomer, grad=True)
MONOMER_ENERGIES['co2'] = V_intra.numpy()[0, 0]

In [ ]:
np.save('MONOMER_ENERGIES', MONOMER_ENERGIES)